## Simple Electrical Grid

<table>
<tr>
<th> Power Generation </th>
<th> Load Points (Pd) </th>
</tr>
<tr>
<td>

| **ID** | **Cost** | **Generation Limit** |
| ------- | ------| ------- |
| 0 | 0.10 | 20 kW |
| 1 | 0.05 | 10 kW |
| 2 | 0.30 | 40 kW |
| 3 | 0.40 | 50 kW |
| 4 | 0.01 | 5 kW |

</td>
<td>

| **ID** | **Load Demand**|
| -- | -- |
| 0 | 50kW|
| 1 | 20kW|
| 2 | 30kW|

</td>
</tr>
</table>

### Formulation

$\text{min} \sum_{i_g=0}^{4}{C_g(i_g)\cdot P_g(i_g)}$

$\sum_{i_g=0}^{4}{P_g(i_g)} ==  \sum_{i_c=0}^{2}{P_d(i_d)}\quad ←\quad \text{Sum of Generation == Sum of Demand}\\$
$P_d(0) \le P_g(0) + P_g(3)\quad ← \quad \text{Only generators 0 and 3 can provide power to load point 0} \\
P_g(i_g) \ge 0,\ \forall\ i_g \quad ← \quad \text{Cannot generate negative Production}\\
P_g(i_g) \le L_g(i_g),\ \forall\ i_g \quad ← \quad \text{Production should be below Plant Limit}$


$i_g \rightarrow \text{Identifier for Generator }g\\
i_d \rightarrow \text{Identifier for Load Point }d\\
C_g \rightarrow \text{Cost of Generation}\\
P_g \rightarrow \text{Power Generated}\\
L_g \rightarrow \text{Production Limit for Generator}$

*Nomenclature is weird because the index seems to be inside parentheses. Maybe it refers to array notation.*

In [14]:
import pandas as pd
df_plants = pd.DataFrame({'id':[0,1,2,3,4], 'Cost':[0.1,0.05,0.3,0.4,0.01],'Limit':[20,10,40,50,5]})
df_demand = pd.DataFrame({'id':[0,1,2],'Demand':[50,20,30]})

# Implement in Pyomo

In [15]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [16]:
n_plants = len(df_plants)

In [17]:
model = pyo.ConcreteModel()
model.Pg = pyo.Var(range(n_plants),bounds=(0,None))

In [18]:
# Build constraint related to total production and total demand
Pg_sum = sum([model.Pg[i] for i in range(n_plants)])
Pd_sum = df_demand['Demand'].sum() 
model.balance = pyo.Constraint(expr = Pg_sum == Pd_sum)
print(f"{Pg_sum} should equal {Pd_sum}.")

Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4] should equal 100.


In [19]:
# Build constraint on load point 0
model.cond0 = pyo.Constraint(expr = model.Pg[0] + model.Pg[3] >= df_demand['Demand'][0])

In [20]:
# Build constraint on Limit per Powerplant
for i in range(n_plants):
    model.Pg[i].upper = df_plants['Limit'][i]

In [21]:
# Revise status of Variables
model.Pg.pprint()

Pg : Size=5, Index={0, 1, 2, 3, 4}
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :    20 : False :  True :  Reals
      1 :     0 :  None :    10 : False :  True :  Reals
      2 :     0 :  None :    40 : False :  True :  Reals
      3 :     0 :  None :    50 : False :  True :  Reals
      4 :     0 :  None :     5 : False :  True :  Reals


In [22]:
# Define Objective Function
objective = sum([model.Pg[i]*df_plants['Cost'][i] for i in range(n_plants)])
model.obj = pyo.Objective(expr = objective)

In [23]:
opt = SolverFactory('gurobi')
results = opt.solve(model)

df_plants['Pg'] = [pyo.value(model.Pg[i]) for i in range(n_plants)]
df_plants

,id,Cost,Limit,Pg
0,0,0.10,20,20.0
1,1,0.05,10,10.0
2,2,0.30,40,35.0
3,3,0.40,50,30.0
4,4,0.01,5,5.0


In [26]:
print(results)


Problem: 
- Name: x1
  Lower bound: 25.05
  Upper bound: 25.05
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 5
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 5
  Number of nonzeros: 7
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.003999948501586914
  Error rc: 0
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

